# LangChain 시작하기 - Model I/O (모델 입출력)

LangChain의 가장 기초적인 구성 요소인 **Model I/O**를 실습합니다.
LLM을 호출하고, 입력을 템플릿화하며, 출력을 원하는 형태로 파싱하는 전체 파이프라인을 익힙니다.

**학습 목표:**
1. **Model (모델):** OpenAI, HuggingFace 등 다양한 LLM을 LangChain으로 호출하는 방법 익히기
2. **Prompt (프롬프트):** `PromptTemplate`을 사용하여 동적이고 재사용 가능한 프롬프트 만들기
3. **Output Parser (출력 파서):** LLM의 텍스트 응답을 리스트, JSON, Pydantic 객체 등 구조화된 데이터로 변환하기

### 1. 환경 설정 (Environment Setup)

실습에 필요한 라이브러리를 설치하고, API 키를 로드합니다.
- `langchain-openai`: OpenAI 모델 연동
- `python-dotenv`: 환경변수(.env) 로드

In [ ]:
%pip install langchain langchain-community langchain-openai langchain-huggingface python-dotenv -qqq

### 2. LangSmith 설정 (선택 사항)
**LangSmith**는 LangChain 애플리케이션의 실행 과정을 추적(Tracing)하고 디버깅할 수 있는 도구입니다.
복잡한 체인(Chain)이 어떻게 동작하는지 시각적으로 확인할 수 있어 매우 유용합니다.

In [ ]:
from dotenv import load_dotenv  # .env 파일에서 환경변수를 불러오는 라이브러리
import os                       # 운영체제 환경변수 제어용

# 1. .env 파일 로드 (API Key 보안 관리)
load_dotenv()

# 2. OpenAI API Key 설정
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")

# 3. LangSmith Tracing 설정 (실행 로그 추적)
os.environ["LANGSMITH_TRACING"] = 'true'
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'         # 프로젝트명 (본인 프로젝트로 변경 가능)
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")

---### 3. Model (모델)

LangChain에서는 다양한 LLM(OpenAI, Anthropic, HuggingFace 등)을 **공통된 인터페이스**로 사용할 수 있습니다.
가장 많이 사용되는 `ChatOpenAI`를 실습해 봅니다.

In [ ]:
from langchain_openai import ChatOpenAI  # OpenAI 채팅 모델 클래스

# 1. 모델 객체 생성 (gpt-4o-mini 모델 사용)
# temperature=0 : 매번 같은 답변을 하도록 설정 (일관성)
# temperature=1 : 창의적이고 다양한 답변 유도
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.7)

# 2. invoke() 메서드로 LLM 호출
# invoke 메서드는 질문(문자열)을 입력받아 AI 응답(AIMessage)을 반환합니다.
response = llm.invoke('프랑스의 여행지 추천해줘.')

# 3. 결과 확인 (content 속성에 실제 텍스트가 들어있습니다)
print(response.content)

#### 📘 모델 추상화 (`init_chat_model`)
특정 클래스(`ChatOpenAI`)에 의존하지 않고, 모델 이름만으로 객체를 생성할 수 있는 팩토리 함수입니다.
나중에 모델을 GPT에서 Claude나 Llama로 바꿀 때 코드 수정이 최소화됩니다.

In [ ]:
from langchain.chat_models import init_chat_model

# 'provider:model_name' 형식으로 지정하여 모델 생성
llm_gpt = init_chat_model('openai:gpt-4o-mini', temperature=1)

result = llm_gpt.invoke('핀란드의 여행지 추천해줘.')
print(result.content)

---### 4. Prompt Template (프롬프트 템플릿)

LLM에게 보낼 질문을 매번 처음부터 작성하는 것은 비효율적입니다.
**PromptTemplate**을 사용하면 구멍 뚫린 틀(Template)을 만들어 두고, 상황에 따라 내용만 바꿔끼울 수 있습니다.

- `PromptTemplate`: 문자열 완성형 템플릿
- `ChatPromptTemplate`: 대화형(System, User) 메시지 템플릿

In [ ]:
from langchain_core.prompts import PromptTemplate

# 1. 템플릿 정의 ({product} 부분이 변수입니다)
template_str = '{product}를 홍보하기 위한 참신한 문구를 작성해 줘.'
prompt = PromptTemplate.from_template(template_str)

# 2. format() 메서드로 변수 채우기
filled_prompt_1 = prompt.format(product='전기차')
filled_prompt_2 = prompt.format(product='카메라')

print(f"완성된 프롬프트 1: {filled_prompt_1}")
print(f"완성된 프롬프트 2: {filled_prompt_2}")

# 3. LLM에 전달하기
# invoke()에 딕셔너리 형태로 변수를 바로 전달할 수도 있습니다.
# (내부적으로 format이 실행되고 모델로 전달됩니다 - Chain 사용 시)
ai_msg = llm.invoke(filled_prompt_1)
print("\n[AI 응답]:\n", ai_msg.content)

---### 5. Output Parser (출력 파서)

LLM은 기본적으로 장문의 줄글(String)을 반환합니다.
하지만 개발 과정에서는 **리스트(List)**나 **JSON** 같은 구조화된 데이터가 필요할 때가 많습니다.

**주요 파서:**
- `CommaSeparatedListOutputParser`: 콤마(,)로 구분된 텍스트를 리스트로 변환
- `JsonOutputParser`: JSON 형태의 텍스트를 파이썬 딕셔너리로 변환
- `PydanticOutputParser`: Pydantic 스키마에 맞춰 엄격하게 검증된 객체로 변환

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# 1. 파서 객체 생성
list_parser = CommaSeparatedListOutputParser()

# 2. 지시사항(Instruction) 확인
# 파서가 LLM에게 "이렇게 답해라"라고 지시할 문구를 자동으로 만들어줍니다.
format_instruction = list_parser.get_format_instructions()
print("[파서 지시사항]:", format_instruction)

# 3. 프롬프트에 지시사항 포함시키기
prompt = PromptTemplate(
    template='{subject} 팀 {n}개를 나열해주세요.\n{format_instruction}',
    input_variables=['subject', 'n'],
    partial_variables={'format_instruction': format_instruction} # 항상 포함될 변수
)

# 4. 체인 실행 (Prompt -> LLM -> Parser)
# | (Pipe) 연산자로 흐름을 연결합니다 (LCEL)
chain = prompt | llm | list_parser

result = chain.invoke({'subject': '한국 프로야구', 'n': 5})

print(f"\n[결과 타입]: {type(result)}")
print(f"[결과 값]: {result}")

#### 📘 JSON 파서 실습
데이터베이스에 저장하거나 API로 전달하기 위해 JSON 형식이 필요할 때 사용합니다.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# 1. 결과값의 구조(Schema)를 Pydantic으로 정의 (권장 사항)
class Book(BaseModel):
    title: str = Field(description="책 제목")
    author: str = Field(description="저자")
    year: int = Field(description="출판 연도")

# 2. 파서 생성
json_parser = JsonOutputParser(pydantic_object=Book)

# 3. 프롬프트 구성
prompt = PromptTemplate(
    template="'{subject}' 주제의 베스트셀러 책 한 권을 추천해줘.\n{format_instruction}",
    input_variables=["subject"],
    partial_variables={"format_instruction": json_parser.get_format_instructions()}
)

# 4. 체인 실행
chain = prompt | llm | json_parser

result = chain.invoke({"subject": "인공지능"})

print(f"[결과 타입]: {type(result)}")  # dict 타입이어야 함
print(json.dumps(result, indent=2, ensure_ascii=False))

---### 6. 정리 (Summary)

- **Model**: `ChatOpenAI` 등을 통해 LLM을 호출합니다.
- **PromptTemplate**: 입력값을 동적으로 채울 수 있는 재사용 가능한 틀입니다.
- **OutputParser**: LLM의 줄글 출력을 리스트나 딕셔너리로 변환하여 프로그램에서 쓰기 좋게 만듭니다.
- **LCEL (`|`)**: `Prompt | Model | Parser` 형태로 우아하게 파이프라인을 구축합니다.